### Import libraries

In [34]:
from IPython.core.display import display
from src.utils.preprocessing import preprocessing_pipeline
from src.utils.get_data import import_data, split_experts
from src.utils.train import hyperparameter_tuning_cv
from src.utils.config import *
import pandas as pd
import numpy as np

In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load Data

In [36]:
DATA_PATH = '../../../data'
X_coarse, y_coarse = import_data(DATA_PATH, segmentation_type='coarse',
                                 drop_user_features=False,
                                 drop_expert=True)

In [37]:
# For each expert separately
X_e, y_e = import_data(DATA_PATH, segmentation_type='coarse',
                       drop_user_features=False,
                       drop_expert=False)

In [38]:
display(X_coarse.head())
display(y_coarse.head())

EEPD50_100  EEPD100_150  \
subject                              file_id                            
008ba489-31ad-44d8-856b-fcf72369dc46 0               8.0          8.0   
                                     1               2.0          3.0   
008c1c9e-aeef-40c5-846c-24f1b964f884 0               4.0          4.0   
                                     1               3.0          5.0   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0               6.0          6.0   

                                              EEPD150_200  EEPD200_250  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                7.0          6.0   
                                     1                2.0          3.0   
008c1c9e-aeef-40c5-846c-24f1b964f884 0                4.0          4.0   
                                     1                5.0          6.0   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0                6.0          7.0   

                                              EEPD250_300  EEPD300_350  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                7.0          9.0   
                                     1                2.0          2.0   
008c1c9e-aeef-40c5-846c-24f1b964f884 0                6.0          6.0   
                                     1                4.0          5.0   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0                6.0          7.0   

                                              EEPD350_400  EEPD400_450  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0                7.0          7.0   
                                     1                3.0          3.0   
008c1c9e-aeef-40c5-846c-24f1b964f884 0                5.0          5.0   
                                     1                6.0          5.0   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0                6.0          6.0   

                                              EEPD450_500  EEPD500_550  ...  \
subject                              file_id                            ...   
008ba489-31ad-44d8-856b-fcf72369dc46 0                7.0          8.0  ...   
                                     1                3.0          3.0  ...   
008c1c9e-aeef-40c5-846c-24f1b964f884 0                5.0          5.0  ...   
                                     1                5.0          4.0  ...   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0                8.0          6.0  ...   

                                              PSD_225-425  PSD_450-550  \
subject                              file_id                             
008ba489-31ad-44d8-856b-fcf72369dc46 0           0.314239     0.027049   
                                     1           0.093550     0.107780   
008c1c9e-aeef-40c5-846c-24f1b964f884 0           0.160032     0.034572   
                                     1           0.050611     0.050448   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0           0.264409     0.050780   

                                              PSD_1325-1600  PSD_1600-2000  \
subject                              file_id                                 
008ba489-31ad-44d8-856b-fcf72369dc46 0             0.015518       0.030041   
                                     1             0.013997       0.008304   
008c1c9e-aeef-40c5-846c-24f1b964f884 0             0.017697       0.029626   
                                     1             0.036759       0.036765   
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0             0.020884       0.008068   

                                              PSD_2500-2900  PSD_3100-3700  \
subject                              file_id                                 
008ba489-31ad-44d8-856b-fcf72369dc46 0             0.047090       0.009558   
                                     1             0.045378       0.007341   
008c1c9e-aeef-40c5-846c-24f1b964f884 0       

Label
subject                              file_id       
008ba489-31ad-44d8-856b-fcf72369dc46 0          1.0
                                     1          1.0
008c1c9e-aeef-40c5-846c-24f1b964f884 0          1.0
                                     1          1.0
00bf9f83-2e8f-47cf-a4f2-97f2beceebc1 0          1.0

### Preprocessing

In [39]:
X_coarse = preprocessing_pipeline(X_coarse)

In [40]:
# Save the expert feature for split
expert = X_e['Expert'].copy()
X_e = preprocessing_pipeline(X_e)
X_e['Expert'] = expert.values

### Split expert models

In [41]:
X_e_1, y_e_1, X_e_2, y_e_2, X_e_3, y_e_3 = split_experts(X_e, y_e)

### Grid search

In [42]:
DECISION_METRIC = 'roc_auc_score'
CV_K = 10

#### 1. Logistic regression

In [43]:
log_results = hyperparameter_tuning_cv(model='logistic', data=X_coarse, labels=y_coarse.Label, cv_k=CV_K,
                                       params=LOGISTIC_PARAMS)

display(log_results)

f1_score  roc_auc_score  roc_auc_score_var  \
max_iter oversampling                                               
10000    True          0.428893       0.596055           0.000133   
         False         0.190028       0.535026           0.000087   
100000   True          0.428893       0.596055           0.000133   
         False         0.190028       0.535026           0.000087   

                       accuracy_score  explained_variance_score  
max_iter oversampling                                            
10000    True                0.637791                 -0.795674  
         False               0.726938                 -0.183461  
100000   True                0.637791                 -0.795674  
         False               0.726938                 -0.183461

In [44]:
# Best model parameters

best_log = log_results.iloc[[
    log_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_log)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_iter,oversampling,,,,,
10000,True,0.428893,0.596055,0.000133,0.637791,-0.795674


In [45]:
# for each expert
log_results_1 = hyperparameter_tuning_cv(model='logistic', data=X_e_1, labels=y_e_1, cv_k=CV_K,
                                         params=LOGISTIC_PARAMS)
log_results_2 = hyperparameter_tuning_cv(model='logistic', data=X_e_2, labels=y_e_2, cv_k=CV_K,
                                         params=LOGISTIC_PARAMS)
log_results_3 = hyperparameter_tuning_cv(model='logistic', data=X_e_3, labels=y_e_3, cv_k=CV_K,
                                         params=LOGISTIC_PARAMS)

In [46]:
# Best model parameters

best_log_1 = log_results_1.iloc[[
    log_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_log_2 = log_results_2.iloc[[
    log_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_log_3 = log_results_3.iloc[[
    log_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_log_1, best_log_2, best_log_3)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_iter,oversampling,,,,,
10000,True,0.364467,0.640867,0.000753,0.759283,-0.872209


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_iter,oversampling,,,,,
10000,True,0.266119,0.532979,0.000405,0.642819,-1.320197


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_iter,oversampling,,,,,
10000,False,0.617514,0.641445,0.000357,0.642857,-0.426392


#### 2. Linear Discriminant Analysis

In [47]:
lda_results = hyperparameter_tuning_cv(model='lda', data=X_coarse, labels=y_coarse.Label, cv_k=CV_K,
                                       params=LDA_PARAMS)

display(lda_results)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.428824,0.596361,0.000176,0.639341,-0.790511
False,0.194535,0.536379,0.000041,0.727422,-0.183448


In [48]:
# Best model parameters

best_lda = lda_results.iloc[[
    lda_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_lda)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.428824,0.596361,0.000176,0.639341,-0.790511


In [49]:
# For each expert
lda_results_1 = hyperparameter_tuning_cv(model='lda', data=X_e_1, labels=y_e_1, cv_k=CV_K,
                                         params=LDA_PARAMS)
lda_results_2 = hyperparameter_tuning_cv(model='lda', data=X_e_2, labels=y_e_2, cv_k=CV_K,
                                         params=LDA_PARAMS)
lda_results_3 = hyperparameter_tuning_cv(model='lda', data=X_e_3, labels=y_e_3, cv_k=CV_K,
                                         params=LDA_PARAMS)

In [50]:
best_lda_1 = lda_results_1.iloc[[
    lda_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_lda_2 = lda_results_2.iloc[[
    lda_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_lda_3 = lda_results_3.iloc[[
    lda_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_lda_1, best_lda_2, best_lda_3)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.3739,0.653304,0.000844,0.7443,-0.946999


,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.262352,0.53026,0.000713,0.642287,-1.326226


,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
False,0.612285,0.63845,0.00036,0.64,-0.436563


#### 3. K-nearest Neighbors

In [51]:
knn_results = hyperparameter_tuning_cv(model='knn', data=X_coarse, labels=y_coarse.Label, cv_k=CV_K,
                                       params=KNN_PARAMS)

display(knn_results)

f1_score  roc_auc_score  roc_auc_score_var  \
n_neighbors oversampling                                               
1           True          0.468548       0.619924           0.000200   
            False         0.443651       0.618595           0.000073   
2           True          0.446534       0.611984           0.000293   
            False         0.237103       0.547534           0.000029   
3           True          0.463438       0.604908           0.000138   
            False         0.355250       0.575409           0.000206   
4           True          0.455686       0.606087           0.000178   
            False         0.220397       0.541376           0.000036   
5           True          0.457991       0.593137           0.000082   
            False         0.298973       0.555685           0.000163   
6           True          0.451964       0.595051           0.000202   
            False         0.194692       0.534497           0.000068   
7           True          0.448527       0.579113           0.000156   
            False         0.266912       0.549106           0.000099   
8           True          0.450247       0.588144           0.000106   
            False         0.177174       0.530305           0.000059   
9           True          0.447457       0.575188           0.000169   
            False         0.248161       0.546362           0.000109   
10          True          0.444952       0.578855           0.000139   
            False         0.171900       0.533075           0.000074   
11          True          0.443587       0.567835           0.000171   
            False         0.219056       0.540778           0.000129   
12          True          0.442935       0.573357           0.000186   
            False         0.154298       0.528581           0.000016   
13          True          0.440833       0.562844           0.000152   
            False         0.183887       0.531617           0.000047   
14          True          0.441023       0.568531           0.000212   
            False         0.130493       0.522557           0.000026   
15          True          0.440969       0.561430           0.000263   
            False         0.157778       0.526684           0.000053   

                          accuracy_score  explained_variance_score  
n_neighbors oversampling                                            
1           True                0.612791                 -0.783883  
            False               0.694671                 -0.551187  
2           True                0.654651                 -0.717203  
            False               0.727616                 -0.218966  
3           True                0.550388                 -0.827344  
            False               0.697674                 -0.512557  
4           True                0.592636                 -0.844881  
            False               0.724322                 -0.228755  
5           True                0.514438                 -0.816936  
            False               0.703198                 -0.442241  
6           True                0.553488                 -0.880655  
            False               0.723643                 -0.211952  
7           True                0.490795                 -0.833279  
            False               0.712306                 -0.354809  
8           True                0.526938                 -0.876210  
            False               0.724031                 -0.194790  
9           True                0.477035                 -0.809594  
            False               0.718508                 -0.294845  
10          True                0.505911                 -0.879599  
            False               0.731202                 -0.135164  
11          True                0.461240                 -0.796709  
            False               0.723740                 -0.231875  
12          True                0.489341                 -0.86581

In [52]:
# Best model parameters

best_knn = knn_results.iloc[[
    knn_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_knn)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,,
1,True,0.468548,0.619924,0.0002,0.612791,-0.783883


In [53]:
knn_results_1 = hyperparameter_tuning_cv(
    model='knn', data=X_e_1, labels=y_e_1, cv_k=CV_K, params=KNN_PARAMS)
knn_results_2 = hyperparameter_tuning_cv(
    model='knn', data=X_e_2, labels=y_e_2, cv_k=CV_K, params=KNN_PARAMS)
knn_results_3 = hyperparameter_tuning_cv(
    model='knn', data=X_e_3, labels=y_e_3, cv_k=CV_K, params=KNN_PARAMS)

In [54]:
display(knn_results_1, knn_results_2, knn_results_3)

f1_score  roc_auc_score  roc_auc_score_var  \
n_neighbors oversampling                                               
1           True          0.367133       0.659582           0.001366   
            False         0.400402       0.652262           0.000771   
2           True          0.380025       0.661933           0.001472   
            False         0.276758       0.579811           0.000205   
3           True          0.335296       0.639032           0.000628   
            False         0.311753       0.597135           0.001123   
4           True          0.343695       0.644055           0.001222   
            False         0.232462       0.563138           0.000409   
5           True          0.326485       0.633212           0.000284   
            False         0.272671       0.577158           0.000489   
6           True          0.330606       0.635207           0.000769   
            False         0.147586       0.533953           0.000145   
7           True          0.315262       0.619026           0.000439   
            False         0.193643       0.546036           0.000414   
8           True          0.318258       0.621294           0.000472   
            False         0.097344       0.518502           0.000295   
9           True          0.304461       0.605206           0.000601   
            False         0.127041       0.525590           0.000412   
10          True          0.313706       0.616968           0.000719   
            False         0.069969       0.511275           0.000116   
11          True          0.308742       0.611834           0.000672   
            False         0.103410       0.518570           0.000270   
12          True          0.315401       0.619542           0.000743   
            False         0.070415       0.512972           0.000079   
13          True          0.305843       0.608427           0.001167   
            False         0.076441       0.512546           0.000117   
14          True          0.311370       0.615232           0.000969   
            False         0.057055       0.510175           0.000045   
15          True          0.303893       0.605532           0.001243   
            False         0.070471       0.512572           0.000119   

                          accuracy_score  explained_variance_score  
n_neighbors oversampling                                            
1           True                0.691857                 -1.161129  
            False               0.818893                 -0.456615  
2           True                0.738111                 -0.959942  
            False               0.857003                 -0.077140  
3           True                0.592508                 -1.442038  
            False               0.825733                 -0.386194  
4           True                0.634528                 -1.368099  
            False               0.852443                 -0.107926  
5           True                0.548860                 -1.470596  
            False               0.840065                 -0.245835  
6           True                0.586645                 -1.461857  
            False               0.847883                 -0.120220  
7           True                0.509772                 -1.487803  
            False               0.838762                 -0.231878  
8           True                0.542997                 -1.512913  
            False               0.844300                 -0.141726  
9           True                0.486319                 -1.513521  
            False               0.840717                 -0.188317  
10          True                0.520521                 -1.512050  
            False               0.845277                 -0.117974  
11          True                0.480456                 -1.467162  
            False               0.841368                 -0.172045  
12          True                0.513029                 -1.48809

f1_score  roc_auc_score  roc_auc_score_var  \
n_neighbors oversampling                                               
1           True          0.328676       0.581350       3.960881e-04   
            False         0.298203       0.575845       7.982115e-04   
2           True          0.317945       0.577239       6.041645e-04   
            False         0.063006       0.506914       6.667467e-05   
3           True          0.312659       0.556285       4.495218e-04   
            False         0.157820       0.521840       2.462170e-04   
4           True          0.309420       0.557752       3.271568e-04   
            False         0.050770       0.507084       2.937483e-05   
5           True          0.314975       0.552768       5.167644e-04   
            False         0.092258       0.510814       2.464640e-04   
6           True          0.310015       0.551600       5.875720e-04   
            False         0.027145       0.503581       5.495883e-05   
7           True          0.316296       0.551228       5.421585e-04   
            False         0.050756       0.506933       3.020403e-05   
8           True          0.313814       0.551930       5.134314e-04   
            False         0.010898       0.501665       2.984561e-05   
9           True          0.312861       0.543846       7.348557e-04   
            False         0.016494       0.501050       4.753434e-05   
10          True          0.309635       0.543009       6.235357e-04   
            False         0.008231       0.500986       2.769892e-05   
11          True          0.310523       0.538718       4.857862e-04   
            False         0.024774       0.504804       3.951888e-05   
12          True          0.309345       0.540235       3.691893e-04   
            False         0.000000       0.499357       6.196888e-07   
13          True          0.314508       0.543195       2.835947e-04   
            False         0.005455       0.500273       1.007247e-05   
14          True          0.313151       0.544657       6.630335e-04   
            False         0.000000       0.499678       4.136857e-07   
15          True          0.311066       0.536103       4.515280e-04   
            False         0.005496       0.500753       1.171318e-05   

                          accuracy_score  explained_variance_score  
n_neighbors oversampling                                            
1           True                0.613564                 -1.305274  
            False               0.757181                 -0.651486  
2           True                0.667021                 -1.149476  
            False               0.809309                 -0.135137  
3           True                0.519415                 -1.472783  
            False               0.784840                 -0.395469  
4           True                0.565160                 -1.452731  
            False               0.814628                 -0.082014  
5           True                0.464362                 -1.419365  
            False               0.802128                 -0.215378  
6           True                0.505319                 -1.484661  
            False               0.817819                 -0.042941  
7           True                0.436436                 -1.351408  
            False               0.814362                 -0.084946  
8           True                0.472606                 -1.437224  
            False               0.820213                 -0.013120  
9           True                0.411436                 -1.311670  
            False               0.817287                 -0.043200  
10          True                0.441755                 -1.415740  
            False               0.819947                 -0.014415  
11          True                0.393883                 -1.275791  
            False               0.820745                 -0.014973  
12          True                0.421809                 -1.37251

f1_score  roc_auc_score  roc_auc_score_var  \
n_neighbors oversampling                                               
1           True          0.645243       0.655031           0.000229   
            False         0.639512       0.656636           0.000233   
2           True          0.540837       0.632592           0.000380   
            False         0.498572       0.621683           0.000380   
3           True          0.639230       0.642514           0.000358   
            False         0.620053       0.643324           0.000468   
4           True          0.573813       0.634285           0.000413   
            False         0.519111       0.621569           0.000272   
5           True          0.632202       0.637702           0.000275   
            False         0.606441       0.642106           0.000341   
6           True          0.581326       0.630744           0.000352   
            False         0.528126       0.622370           0.000326   
7           True          0.631164       0.636454           0.000297   
            False         0.594835       0.634984           0.000433   
8           True          0.596180       0.637161           0.000327   
            False         0.528900       0.622066           0.000216   
9           True          0.631989       0.636288           0.000417   
            False         0.591920       0.634225           0.000631   
10          True          0.588135       0.627912           0.000385   
            False         0.540635       0.625507           0.000538   
11          True          0.621105       0.626924           0.000511   
            False         0.588894       0.634644           0.000739   
12          True          0.598369       0.631804           0.000315   
            False         0.546802       0.631050           0.000682   
13          True          0.624808       0.628529           0.000415   
            False         0.587351       0.635697           0.000475   
14          True          0.607376       0.636070           0.000141   
            False         0.548116       0.632746           0.000312   
15          True          0.621350       0.625138           0.000305   
            False         0.581062       0.633119           0.000318   

                          accuracy_score  explained_variance_score  
n_neighbors oversampling                                            
1           True                0.654571                 -0.382123  
            False               0.656857                 -0.372366  
2           True                0.640000                 -0.319298  
            False               0.630286                 -0.278626  
3           True                0.641429                 -0.431136  
            False               0.643429                 -0.420846  
4           True                0.639429                 -0.395144  
            False               0.628857                 -0.342830  
5           True                0.637429                 -0.450162  
            False               0.644286                 -0.410275  
6           True                0.635143                 -0.432048  
            False               0.629143                 -0.361548  
7           True                0.636286                 -0.455302  
            False               0.637143                 -0.433212  
8           True                0.640857                 -0.419783  
            False               0.628857                 -0.365608  
9           True                0.636286                 -0.452982  
            False               0.637143                 -0.431395  
10          True                0.632000                 -0.458051  
            False               0.631714                 -0.372427  
11          True                0.627143                 -0.491609  
            False               0.637429                 -0.423666  
12          True                0.634857                 -0.45342

In [55]:
best_knn_1 = knn_results_1.iloc[[
    knn_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_knn_2 = knn_results_2.iloc[[
    knn_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_knn_3 = knn_results_3.iloc[[
    knn_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_knn_1, best_knn_2, best_knn_3)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,,
2,True,0.380025,0.661933,0.001472,0.738111,-0.959942


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,,
1,True,0.328676,0.58135,0.000396,0.613564,-1.305274


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
n_neighbors,oversampling,,,,,
1,False,0.639512,0.656636,0.000233,0.656857,-0.372366


#### 4. Support Vector Classifier

In [56]:
svc_results = hyperparameter_tuning_cv(model='svc', data=X_coarse, labels=y_coarse.Label, cv_k=CV_K,
                                       params=SVC_PARAMS)

display(svc_results)

f1_score  roc_auc_score  roc_auc_score_var  \
kernel gamma oversampling                                               
linear 0.10  True          0.424076       0.592715           0.000236   
             False         0.000000       0.500000           0.000000   
rbf    0.10  True          0.138128       0.531521           0.000025   
             False         0.011451       0.502106           0.000004   
linear 0.01  True          0.424076       0.592715           0.000236   
             False         0.000000       0.500000           0.000000   
rbf    0.01  True          0.434941       0.606863           0.000248   
             False         0.006488       0.501369           0.000002   

                           accuracy_score  explained_variance_score  
kernel gamma oversampling                                            
linear 0.10  True                0.636628                 -0.804580  
             False               0.731008                  0.000000  
rbf    0.10  True                0.741376                 -0.027409  
             False               0.731395                 -0.005394  
linear 0.01  True                0.636628                 -0.804580  
             False               0.731008                  0.000000  
rbf    0.01  True                0.665504                 -0.683980  
             False               0.731492                 -0.000926

In [57]:
# Best model parameters

best_svc = svc_results.iloc[[
    svc_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_svc)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,,
rbf,0.01,True,0.434941,0.606863,0.000248,0.665504,-0.68398


In [58]:
svc_results_1 = hyperparameter_tuning_cv(
    model='svc', data=X_e_1, labels=y_e_1, cv_k=CV_K, params=SVC_PARAMS)
svc_results_2 = hyperparameter_tuning_cv(
    model='svc', data=X_e_2, labels=y_e_2, cv_k=CV_K, params=SVC_PARAMS)
svc_results_3 = hyperparameter_tuning_cv(
    model='svc', data=X_e_3, labels=y_e_3, cv_k=CV_K, params=SVC_PARAMS)

In [59]:
best_svc_1 = svc_results_1.iloc[[
    svc_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_svc_2 = svc_results_2.iloc[[
    svc_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_svc_3 = svc_results_3.iloc[[
    svc_results_3.reset_index()[DECISION_METRIC].idxmax()]]


display(best_svc_1, best_svc_2, best_svc_3)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,,
linear,0.1,True,0.374145,0.649842,0.000417,0.758958,-0.866928


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,,
linear,0.1,True,0.263785,0.5304,0.000282,0.637766,-1.342262


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
kernel,gamma,oversampling,,,,,
rbf,0.01,False,0.63429,0.663262,0.000192,0.664857,-0.332676


#### 5. Naive Bayes Classifier

In [60]:
nb_results = hyperparameter_tuning_cv(model='naive_bayes', data=X_coarse, labels=y_coarse.Label, cv_k=CV_K,
                                      params=NAIVE_BAYES_PARAMS)

display(nb_results)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.097475,0.514818,0.000866,0.688178,-0.220509
False,0.091870,0.514923,0.000948,0.714244,-0.178692


In [61]:
# Best model parameters

best_nb = nb_results.iloc[[nb_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_nb)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
False,0.09187,0.514923,0.000948,0.714244,-0.178692


In [62]:
nb_results_1 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_1, labels=y_e_1, cv_k=CV_K,
    params=NAIVE_BAYES_PARAMS)
nb_results_2 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_2, labels=y_e_2, cv_k=CV_K,
    params=NAIVE_BAYES_PARAMS)
nb_results_3 = hyperparameter_tuning_cv(
    model='naive_bayes', data=X_e_3, labels=y_e_3, cv_k=CV_K,
    params=NAIVE_BAYES_PARAMS)

In [63]:
best_nb_1 = nb_results_1.iloc[[
    nb_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_nb_2 = nb_results_2.iloc[[
    nb_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_nb_3 = nb_results_3.iloc[[
    nb_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_nb_1, best_nb_2, best_nb_3)

,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
True,0.301947,0.594561,0.000978,0.727687,-1.095128


,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
False,0.294778,0.532091,0.001164,0.50133,-1.537017


,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
oversampling,,,,,
False,0.18167,0.520906,0.001138,0.540286,-0.237237


#### 6. Decision Tree

In [64]:
dt_results = hyperparameter_tuning_cv(model='decision_tree', data=X_coarse, labels=y_coarse.Label, cv_k=CV_K,
                                      params=DECISION_TREE_PARAMS)

display(dt_results)

f1_score  roc_auc_score  roc_auc_score_var  \
max_depth oversampling                                               
3         True          0.351964       0.529400           0.000300   
          False         0.064378       0.508511           0.000115   
5         True          0.374372       0.557682           0.000213   
          False         0.197162       0.527772           0.000360   
7         True          0.385628       0.563532           0.000274   
          False         0.263464       0.538929           0.000207   

                        accuracy_score  explained_variance_score  
max_depth oversampling                                            
3         True                0.550291                 -1.008510  
          False               0.725872                 -0.090567  
5         True                0.612403                 -0.895144  
          False               0.707752                 -0.330389  
7         True                0.617248                 -0.905391  
          False               0.690698                 -0.495519

In [65]:
# Best model parameters

best_dt = dt_results.iloc[[dt_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_dt)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,oversampling,,,,,
7,True,0.385628,0.563532,0.000274,0.617248,-0.905391


In [66]:
dt_results_1 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_1, labels=y_e_1, cv_k=CV_K, params=DECISION_TREE_PARAMS)
dt_results_2 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_2, labels=y_e_2, cv_k=CV_K, params=DECISION_TREE_PARAMS)
dt_results_3 = hyperparameter_tuning_cv(
    model='decision_tree', data=X_e_3, labels=y_e_3, cv_k=CV_K, params=DECISION_TREE_PARAMS)

In [67]:
best_dt_1 = dt_results_1.iloc[[
    dt_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_dt_2 = dt_results_2.iloc[[
    dt_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_dt_3 = dt_results_3.iloc[[
    dt_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_dt_1, best_dt_2, best_dt_3)

,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,oversampling,,,,,
3,True,0.342055,0.631774,0.001776,0.694463,-1.194066


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,oversampling,,,,,
5,True,0.245803,0.522685,0.001978,0.646277,-1.25654


,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,oversampling,,,,,
3,True,0.537808,0.607115,0.000258,0.613143,-0.479072


#### 7. Random Forest

In [68]:
rf_results = hyperparameter_tuning_cv(model='random_forest', data=X_coarse, labels=y_coarse.Label, cv_k=CV_K,
                                      params=RANDOM_FOREST_PARAMS)

display(rf_results)

f1_score  roc_auc_score  \
max_depth n_estimators oversampling                            
3         3            True          0.395191       0.554517   
                       False         0.036141       0.505414   
          5            True          0.413601       0.574476   
                       False         0.014301       0.502764   
          7            True          0.396937       0.565995   
                       False         0.002974       0.500484   
5         3            True          0.394964       0.563284   
                       False         0.110989       0.512183   
          5            True          0.404304       0.573236   
                       False         0.072997       0.509164   
          7            True          0.412767       0.581815   
                       False         0.055596       0.509231   
7         3            True          0.383368       0.557076   
                       False         0.187998       0.527955   
          5            True          0.390750       0.566293   
                       False         0.149772       0.521903   
          7            True          0.395267       0.574663   
                       False         0.129664       0.521362   

                                     roc_auc_score_var  accuracy_score  \
max_depth n_estimators oversampling                                      
3         3            True                   0.000348        0.566667   
                       False                  0.000033        0.730136   
          5            True                   0.000263        0.592733   
                       False                  0.000006        0.731686   
          7            True                   0.000377        0.600678   
                       False                  0.000002        0.731008   
5         3            True                   0.000183        0.595930   
                       False                  0.000100        0.717926   
          5            True                   0.000369        0.611337   
                       False                  0.000089        0.725484   
          7            True                   0.000253        0.621221   
                       False                  0.000039        0.730620   
7         3            True                   0.000102        0.600678   
                       False                  0.000090        0.714535   
          5            True                   0.000194        0.615891   
                       False                  0.000051        0.720736   
          7            True                   0.000109        0.635271   
                       False                  0.000050        0.727132   

                                     explained_variance_score  
max_depth n_estimators oversampling                            
3         3            True                         -1.033337  
                       False                        -0.033573  
          5            True                         -0.942366  
                       False                        -0.005174  
          7            True                         -0.946844  
                       False                        -0.002058  
5         3            True                         -0.960309  
                       False                        -0.187325  
          5            True                         -0.907718  
                       False                        -0.098061  
          7            True                         -0.863286  
                       False                        -0.044763  
7         3            True                         -0.964930  
                       False                        -0.274259  
          5            True                         -0.909431  
                       False                        -0.196841  
          7            True                         -0.833404  
                       False                        -0.1310

In [69]:
# Best model parameters

best_rf = rf_results.iloc[[rf_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_rf)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
5,7,True,0.412767,0.581815,0.000253,0.621221,-0.863286


In [70]:
rf_results_1 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_1, labels=y_e_1, cv_k=CV_K, params=RANDOM_FOREST_PARAMS)
rf_results_2 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_2, labels=y_e_2, cv_k=CV_K, params=RANDOM_FOREST_PARAMS)
rf_results_3 = hyperparameter_tuning_cv(
    model='random_forest', data=X_e_3, labels=y_e_3, cv_k=CV_K, params=RANDOM_FOREST_PARAMS)

In [71]:
best_rf_1 = rf_results_1.iloc[[
    rf_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_rf_2 = rf_results_2.iloc[[
    rf_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_rf_3 = rf_results_3.iloc[[
    rf_results_3.reset_index()[DECISION_METRIC].idxmax()]]

display(best_rf_1, best_rf_2, best_rf_3)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
5,7,True,0.382291,0.655406,0.000614,0.767427,-0.813035


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
7,5,True,0.281542,0.549249,0.0011,0.669415,-1.172626


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
7,7,False,0.608261,0.637357,0.000452,0.637714,-0.437479


#### 8. Gradient Boosting

In [72]:
gb_results = hyperparameter_tuning_cv(model='gradient_boosting', data=X_coarse, labels=y_coarse.Label, cv_k=CV_K,
                                      params=GRADIENT_BOOSTING_PARAMS)

display(gb_results)

f1_score  roc_auc_score  \
max_depth n_estimators oversampling                            
3         3            True          0.378887       0.556529   
                       False         0.000000       0.500000   
          5            True          0.388957       0.564662   
                       False         0.000000       0.499934   
          7            True          0.396810       0.571216   
                       False         0.000000       0.500000   
5         3            True          0.378700       0.569376   
                       False         0.000000       0.500000   
          5            True          0.379600       0.570760   
                       False         0.005705       0.500778   
          7            True          0.383914       0.575123   
                       False         0.014842       0.502064   
7         3            True          0.388423       0.575131   
                       False         0.000699       0.500109   
          5            True          0.398834       0.587663   
                       False         0.033342       0.504689   
          7            True          0.392486       0.585594   
                       False         0.066559       0.510893   

                                     roc_auc_score_var  accuracy_score  \
max_depth n_estimators oversampling                                      
3         3            True               3.579595e-04        0.605233   
                       False              0.000000e+00        0.731008   
          5            True               4.669334e-04        0.613275   
                       False              3.968191e-08        0.730911   
          7            True               6.020635e-04        0.619089   
                       False              0.000000e+00        0.731008   
5         3            True               3.151276e-04        0.641376   
                       False              0.000000e+00        0.731008   
          5            True               2.076803e-04        0.648062   
                       False              1.445306e-06        0.730814   
          7            True               3.980905e-04        0.654748   
                       False              7.809781e-06        0.730523   
7         3            True               2.297664e-04        0.649322   
                       False              1.073442e-07        0.731008   
          5            True               5.231089e-04        0.670446   
                       False              1.486276e-05        0.729748   
          7            True               2.449183e-04        0.674709   
                       False              2.201683e-05        0.730329   

                                     explained_variance_score  
max_depth n_estimators oversampling                            
3         3            True                         -0.937375  
                       False                         0.000000  
          5            True                         -0.907505  
                       False                        -0.000756  
          7            True                         -0.885249  
                       False                         0.000000  
5         3            True                         -0.793353  
                       False                         0.000000  
          5            True                         -0.777301  
                       False                        -0.005752  
          7            True                         -0.744608  
                       False                        -0.014797  
7         3            True                         -0.775598  
                       False                        -0.000533  
          5            True                         -0.673514  
                       False                        -0.034589  
          7            True                         -0.650822  
                       False                        -0.0555

In [73]:
# Best model parameters

best_gb = gb_results.iloc[[gb_results.reset_index()[DECISION_METRIC].idxmax()]]

display(best_gb)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
7,5,True,0.398834,0.587663,0.000523,0.670446,-0.673514


In [74]:
gb_results_1 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_1, labels=y_e_1, cv_k=CV_K, params=GRADIENT_BOOSTING_PARAMS)
gb_results_2 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_2, labels=y_e_2, cv_k=CV_K, params=GRADIENT_BOOSTING_PARAMS)
gb_results_3 = hyperparameter_tuning_cv(
    model='gradient_boosting', data=X_e_3, labels=y_e_3, cv_k=CV_K, params=GRADIENT_BOOSTING_PARAMS)

In [75]:
best_gb_1 = gb_results_1.iloc[[
    gb_results_1.reset_index()[DECISION_METRIC].idxmax()]]
best_gb_2 = gb_results_2.iloc[[
    gb_results_2.reset_index()[DECISION_METRIC].idxmax()]]
best_gb_3 = gb_results_3.iloc[[
    gb_results_3.reset_index()[DECISION_METRIC].idxmax()]]
display(best_gb_1, best_gb_2, best_gb_3)

,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
3,3,True,0.380664,0.667278,0.000725,0.715309,-1.057728


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
7,3,True,0.258692,0.546872,0.00045,0.718883,-0.893166


,,,f1_score,roc_auc_score,roc_auc_score_var,accuracy_score,explained_variance_score
max_depth,n_estimators,oversampling,,,,,
5,7,True,0.626931,0.65105,0.000379,0.652,-0.388518


### Results

In [76]:
# display('logistic', best_log)
# display('lda', best_lda)
# display('knn', best_knn)
# display('svc', best_svc)
# display('naive_bayes', best_nb)
# display('decision_tree', best_dt)
# display('random_forest', best_rf)
# display('gradient_boosting', best_gb)

In [77]:
weighted_average = (np.array([len(X_e_1), len(X_e_2), len(X_e_3)]) / len(X_e))

e_log = np.sum(pd.concat([best_log_1, best_log_2, best_log_3]
                         ).roc_auc_score.values * weighted_average)
e_lda = np.sum(pd.concat([best_lda_1, best_lda_2, best_lda_3]
                         ).roc_auc_score.values * weighted_average)
e_knn = np.sum(pd.concat([best_knn_1, best_knn_2, best_knn_3]
                         ).roc_auc_score.values * weighted_average)
e_svc = np.sum(pd.concat([best_svc_1, best_svc_2, best_svc_3]
                         ).roc_auc_score.values * weighted_average)
e_nb = np.sum(pd.concat([best_nb_1, best_nb_2, best_nb_3]
                        ).roc_auc_score.values * weighted_average)
e_dt = np.sum(pd.concat([best_dt_1, best_dt_2, best_dt_3]
                        ).roc_auc_score.values * weighted_average)
e_rf = np.sum(pd.concat([best_rf_1, best_rf_2, best_rf_3]
                        ).roc_auc_score.values * weighted_average)
e_gb = np.sum(pd.concat([best_gb_1, best_gb_2, best_gb_3]
                        ).roc_auc_score.values * weighted_average)

In [78]:
# Results expert features
# First row is Expert 1, second Expert 2, third Expert 3
# display('logistic', pd.concat([best_log_1,best_log_2,best_log_3]))
# display('lda', pd.concat([best_lda_1,best_lda_2,best_lda_3]))
display('knn', pd.concat([best_knn_1, best_knn_2, best_knn_3]))
# display('svc', pd.concat([best_svc_1,best_svc_2,best_svc_3]))
# display('naive_bayes', pd.concat([best_nb_1,best_nb_2,best_nb_3]))
# display('decision_tree', pd.concat([best_dt_1,best_dt_2,best_dt_3]))
# display('random_forest', pd.concat([best_rf_1,best_rf_2,best_rf_3]))
# display('gradient_boosting', pd.concat([best_gb_1,best_gb_2,best_gb_3]))

'knn'

f1_score  roc_auc_score  roc_auc_score_var  \
n_neighbors oversampling                                               
2           True          0.380025       0.661933           0.001472   
1           True          0.328676       0.581350           0.000396   
            False         0.639512       0.656636           0.000233   

                          accuracy_score  explained_variance_score  
n_neighbors oversampling                                            
2           True                0.738111                 -0.959942  
1           True                0.613564                 -1.305274  
            False               0.656857                 -0.372366

In [79]:
results = pd.DataFrame(data={'models': ['logistic', 'lda', 'knn', 'svc', 'naive_bayes', 'decision_tree',
                                        'random_forest', 'gradient_boosting'],
                             'auc_best': [best_log.roc_auc_score.values[0], best_lda.roc_auc_score.values[0],
                                          best_knn.roc_auc_score.values[0], best_svc.roc_auc_score.values[0],
                                          best_nb.roc_auc_score.values[0], best_dt.roc_auc_score.values[0],
                                          best_rf.roc_auc_score.values[0], best_gb.roc_auc_score.values[0]],
                             'auc_expert_weighted': [e_log, e_lda, e_knn, e_svc, e_nb, e_dt, e_rf, e_gb]})

display(results)

,models,auc_best,auc_expert_weighted
0,logistic,0.596055,0.601797
1,lda,0.596361,0.603486
2,knn,0.619924,0.630809
3,svc,0.606863,0.610919
4,naive_bayes,0.514923,0.546859
5,decision_tree,0.563532,0.583713
6,random_forest,0.581815,0.610652
7,gradient_boosting,0.587663,0.617955


### Conclusions


In [84]:
# Save dataframe
results.to_pickle("results_coarse_metadata.pkl")

In [81]:
results.mean(axis=0)

auc_best               0.583392
auc_expert_weighted    0.600774
dtype: float64

In [82]:
results.loc[results['auc_best'].argmax()]

models                      knn
auc_best               0.619924
auc_expert_weighted    0.630809
Name: 2, dtype: object

In [83]:
results.loc[results['auc_expert_weighted'].argmax()]

models                      knn
auc_best               0.619924
auc_expert_weighted    0.630809
Name: 2, dtype: object